In [2]:
require(data.table)
transformationMetrics <<- list()
DATA_PATH = "../input/timeSeries/"#Kaggle computer


In [3]:
file_path = paste(DATA_PATH, 'time_series.csv', sep="")
mts = fread(file_path); head(mts)

In [4]:
#VERIFYING SOME DATES
nRows = nrow(mts)
#nCols = ncol(mts)
as.POSIXct(mts$timestamp[1], origin="1970-01-01 00:00:00.000", tz = "UTC")
as.POSIXct(mts$timestamp[2], origin="1970-01-01 00:00:00.000", tz = "UTC")
as.POSIXct(mts$timestamp[nRows], origin="1970-01-01 00:00:00.000", tz = "UTC")

In [5]:
#SEPARATING TIME, ENDOGENOUS, AND EXOGENOUS VARIABLES
colNms = names(mts)
times = mts[, c("timestamp")]
u_nms = paste("Target", 0:13, sep="_"); #u_nms
x_nms = colNms[-c(1,grep("Target", colNms))]
u = mts[, ..u_nms]#targets
head(u); 

In [6]:
x = mts[, ..x_nms]#covariates
head(x)

In [7]:
#REPLACING Inf CELLS WITH NA
##Are there NAs?
mins = apply(X = u, MARGIN = 2, FUN = min, na.rm=TRUE)
#summary(mins); gc(verbose = FALSE)
mins = apply(X = x, MARGIN = 2, FUN = min, na.rm=TRUE)
#summary(mins); gc(verbose = FALSE)
infCols = which(is.infinite(mins)); infCols
infCols = names(infCols); infCols
InfRows = is.infinite(x[[infCols]])
x[[infCols]][InfRows] = NA#if just one column has Inf
# If more than 1 column have NAs:
# x[,infCols] = apply(X = x[,infCols], MARGIN = 2, 
#               FUN = function(myCol){#myCol = train[,89];#t(row)
#                 infIndexes = which(is.infinite(myCol))
#                 myCol[infIndexes] = NA
#                 return(myCol)
#               }); gc(verbose = FALSE)
# x = as.data.frame(x)
gc();mins = apply(X = x, MARGIN = 2, FUN = min, na.rm=TRUE); summary(mins); gc(verbose = FALSE)
# summary(mins)

In [8]:
#SHIFTING X VARIABLES FOR ACHIEVING 
#MIN EQUAL TO 1000 AND ALLOWING LOG TRANSFORMATION
transformationMetrics$x_min = min(mins)
x = x + 1000 - transformationMetrics$x_min
# [, names(minsLesserThan100)] = apply(X = x[, names(minsLesserThan100)], MARGIN = 2, 
#           FUN = function(myCol){#myCol = train[,89];#t(row)
#             minCol = min(myCol, na.rm=TRUE)
#             myCol = myCol + (100 - minCol)
#             return(myCol)
#           }); gc(verbose = FALSE)
# x = as.data.frame(x)
mins = apply(X = x, MARGIN = 2, FUN = min, na.rm=TRUE); summary(mins); 
x = log(x)
mins = apply(X = x, MARGIN = 2, FUN = min, na.rm=TRUE); summary(mins); 
gc(); 

In [9]:
#TAKING THE DIFFERENTIATED SERIES (TO ACHIEVE LEAK STATIONARITY)
gc(); u = as.data.frame(diff(as.matrix(u)))


In [10]:
performApplyInBlocks = function(x, nBlocks=4, FUN, MARGIN){
  gc(); 
  nCols = ncol(x)
  nBlocks = nBlocks+1
  breaks = as.integer(seq(from=0, to=nCols, length.out = nBlocks))
  if(breaks[nBlocks]!=nCols){breaks[nBlocks+1] = nCols}
  aux =NULL
  for(i in 2:nBlocks){
    begin =  breaks[i-1]+1
    end = breaks[i]
    gc();
    if(MARGIN==2){
      aux = cbind(aux, 
                  apply(X =x[,begin:end], MARGIN = MARGIN, FUN = FUN)
      )
    }
    else if(MARGIN==1){
      aux = rbind(aux, 
                  apply(X =x[begin:end, ], MARGIN = MARGIN, FUN = FUN)
      )
    }
  }
  return(aux)
}
#aux = performApplyInBlocks(x = as.matrix(x[,2:ncol(x)]), nBlocks = 6, FUN = diff, MARGIN = 2)
x = as.data.frame(diff(as.matrix(x[,2:ncol(x)])))
#rm(list=c("trash1", "trash2", "infCols"))
x = cbind(timestamp = times$timestamp[2:length(times$timestamp)], x)
gc(); View(head(x))




In [51]:
#gc()
#mins = apply(X = cbind(u, x), MARGIN = 2, FUN = min, na.rm=TRUE)
#maxs = apply(X = cbind(u, x), MARGIN = 2, FUN = max, na.rm=TRUE)
#cbind(min = summary(mins), max =summary(maxs))

In [52]:
#STUDYING TARGETS TREND
gc()
spearman.p = apply(X = u, MARGIN = 2, 
             FUN = function(y){
               ret = cor.test(x = x$timestamp, y=y, method="spearman")
               return(ret$p.value)
             }
)
#summary(spearman.p)#If min>=.05, we have no trend


In [53]:
#gc()
#spearman.p = apply(X = x, MARGIN = 2, 
#             FUN = function(y){
#               ret = cor.test(x = x$timestamp, y=y, method="spearman")
#               return(ret$p.value)
#             }
#)
#summary(spearman.p)#If min>=.05, we have no trend


In [36]:
#separating train and test sets
nRows = nrow(u)
pTrain = .75
n = round(pTrain*nRows)
#rm("train_mts", "test_mts")
u_train = u[1:n, ]; u_test = u[(n+1):nRows, ]; head(u_train); 

In [35]:
x_train = x[1:n, ]; x_test = x[(n+1):nRows, ]; head(x_train); 

In [37]:
getStandardizedData = function(data, dataNm, means = NULL, sds = NULL){
  gc()
  if(is.null(means)){
    means = colMeans(x = data, na.rm = TRUE)
    transformationMetrics[[dataNm]]$means <<- means
  }
  if(is.null(sds)){
    sds = apply(X = data, MARGIN = 2, FUN = sd, na.rm=TRUE)
    transformationMetrics[[dataNm]]$sds <<- sds
  }
  print(cbind(means=summary(means), sds=summary(sds)))
  gc()
#  nCols = ncol(data)
#  half = round(.5*nCols)#for dealing with memory consumption
  data_st = as.data.frame(scale(x = data, center = means, scale = sds))
  mins = apply(X = data_st, MARGIN = 2, FUN = min, na.rm=TRUE)
  maxs = apply(X = data_st, MARGIN = 2, FUN = max, na.rm=TRUE)
  print(cbind(min = summary(mins), max = summary(maxs)))
  return(data_st)
}
u_train = getStandardizedData(u_train, "u_train", 
                              means = transformationMetrics[["u_train"]]$means, 
                             sds = transformationMetrics[["u_train"]]$sds)
gc(); sds = apply(X = u_train, MARGIN = 2, FUN = sd, na.rm=TRUE); summary(sds)


In [60]:
means = colMeans(u_train, na.rm = TRUE); summary(means)


In [38]:
x_train = getStandardizedData(x_train, "x_train", 
                              means = transformationMetrics[["x_train"]]$means, 
                             sds = transformationMetrics[["x_train"]]$sds)
sds = apply(X = x_train, MARGIN = 2, FUN = sd, na.rm=TRUE); 
means = colMeans(x_train, na.rm = TRUE); cbind(means = summary(means), sds = summary(sds))
gc()

In [ ]:
#ks.p = apply(X = u_train, MARGIN = 2, 
#             FUN = function(x){
#                ret = ks.test(x, 'pnorm', 0, 1)
#                return(ret$p.value)
#              }
#             )
#summary(ks.p)#as expected, normal distribution does not fitt (big data)


In [ ]:
#STUDYING NA cases (IMPUTATION AND DIMENSION REDUCTION)
#getNAsCount = function(sample){#x = train_mts[2,];#t(x)
#  ret = length(which(is.na(sample)))
#  return(ret)
#}
#x_nCols = ncol(x_train)
#x_nColsHalf = round(x_nCols*.5)
#gc(); nNAsPerCol = apply(X = x_train, MARGIN = 2, FUN = getNAsCount)
#gc(); nNAsPerCol = c(nNAsPerCol, apply(X = x_train[,(x_nColsHalf+1):x_nCols], MARGIN = 2, FUN = getNAsCount))
#summary(nNAsPerCol)
#colnames(x_train)[nNAsPerCol==nrow(x_train)]
#colnames(x_train)[nNAsPerCol==0]
getIsNaCol = function(x){
    #nm_x = colnames(x)
    #print(paste(">>>>>>>", nm_x))
    #isNA = is.na(x)
    #after standardization, the mean value of the column is zero 
    #(this is the first step of the imputation method):
    #x[isNA] = 0
    #a new variale indicating where imputation has happened is created
    #this is the second step of the imputation method:
    isNA = as.integer(is.na(x))
    #nm_dummy = paste(nm_x, "isNA", sep="_")
    #ret = as.list(x)
    #ret[[nm_dummy]] = isNA
    #ret = as.data.frame(ret)
    return(isNA)
}
getImputedCol = function(x){
    #isNA = is.na(x)
    #after standardization, the mean value of the column is zero 
    #(this is the first step of the imputation method):
    #x[isNA] = 0
    x_imp = na_ma(x)
    return(x_imp)
}
getImputedAndAugmentedData = function(data){
    data_isNA = as.data.frame(apply(X = data, MARGIN = 2, FUN = getIsNaCol))
    colnames(data_isNA) = paste(colnames(data_isNA), "isNA", sep="_")
    require(imputeTS)
    data_Imputation = as.data.frame(apply(X = data, MARGIN = 2, FUN = getImputedCol))
    augmentedData = cbind(data_Imputation, data_isNA)
    head(augmentedData)
    return(augmentedData)
}
#u_train_NAs = as.data.frame(apply(X = u_train, MARGIN = 2, FUN = getIsNaCol))
#colnames(u_train_NAs) = paste(colnames(u_train_NAs), "isNA", sep="_")
#head(u_train_NAs)
#summary(u_train_NAs)
#head(u_train)
#u_train_imp = as.data.frame(apply(X = u_train, MARGIN = 2, FUN = getZeroImputedCol))
#head(u_train_imp)
getImputedAndAugmentedData(u_train[1:100, 4:9])

In [63]:
##TAKING TRAIN DATASET WITHOUT NAs (FOR IMPUTATION AND DIMENSION REDUCTION)
#x_nRows = nrow(x_train)
#x_nRowsHalf = round(x_nRows*.5)
#gc(); nNAsPerRow = apply(X = x_train, MARGIN = 1, FUN = getNAsCount)
#gc(); nNAsPerRow = c(nNAsPerRow, apply(X = x_train[(x_nRowsHalf+1):x_nRows, ], MARGIN = 1, FUN = getNAsCount))
#summary(nNAsPerRow)#there rows without NA
#noNArowsIndexes = which(nNAsPerRow==0)
#summary(noNArowsIndexes)
#x_train_full = x_train[noNArowsIndexes,]
#View(head(x_train_full))
#nrow(x_train_full)/(24*60*60)#the number of days without NAs


In [64]:
#is there time gaps?
#diffs = diff(noNArowsIndexes)
#cbind(table(diffs))


In [65]:
#studying linear dependency among covariates
comp = prcomp(x_train_full)
s_comp = (summary(comp))
comp_importance = as.data.frame(s_comp$importance)
eingenValuesGreaterThanOne_indexes = which(comp_importance[1,]>1)
comp = prcomp(x_train_full, rank. = max(eingenValuesGreaterThanOne_indexes))
(summary(comp))
gc()
plot(comp)


In [66]:
require(imputeTS)
gc(); u_train = as.data.frame(apply(X = u_train, MARGIN = 2, FUN = na_mean, option="mean"))
#summary(u_train)
gc(); x_train = as.data.frame(apply(X = x_train, MARGIN = 2, FUN = na_mean, option="mean"))
#gc(); aux_train2 = as.data.frame(apply(X = x_train[, (x_nColsHalf+1):x_nCols], MARGIN = 2, FUN = na_interpolation, option="spline"))
#gc(); x_train = cbind(aux_train, aux_train2)


In [68]:
gc()
means = c(colMeans(u_train), colMeans(x_train))
sds = c(apply(X = u_train, MARGIN = 2, FUN = sd), apply(X = x_train, MARGIN = 2, FUN = sd))
cbind(means = means, sds = sds)


In [70]:
# install.packages("MTS")
require("MTS")
mq(u_train,lag=10)


In [1]:
gc()
my_n = nrow(u_train)
#VARXorder(x = u_train[1:my_n, ], exog = x_train[1:my_n, ], maxp = 10, maxm = 10)